In [4]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import os
import zipfile
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
# Path to the zip file containing the dataset
zip_file_path = '/content/data.zip'
# Directory to extract the dataset
extracted_dir = '/content/data/AllImage/AllImage/AllImage/AllImage'
csv_file_path ="Labelss.csv"
# Extracting the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Loading class labels from the CSV file
labels_df = pd.read_csv(csv_file_path)

# Initializing lists to store image data and corresponding labels
images = []
labels = []

# Loading the images and labels from the extracted folder and CSV file
for index, row in labels_df.iterrows():
    img_file = row['image_files']
    img_path = os.path.join(extracted_dir, img_file)
    img = Image.open(img_path)
    images.append(np.array(img))
    labels.append(row['label'])

# Converting lists to numpy arrays
images = np.array(images)
labels = np.array(labels)
print(images)
print(labels)

[[[[ 29  29  27]
   [ 28  28  26]
   [ 26  26  24]
   ...
   [ 39  39  37]
   [ 32  32  30]
   [ 28  28  26]]

  [[ 28  28  26]
   [ 27  27  25]
   [ 25  25  23]
   ...
   [ 38  38  36]
   [ 38  38  36]
   [ 38  38  36]]

  [[ 28  28  26]
   [ 30  30  28]
   [ 31  31  29]
   ...
   [ 34  34  32]
   [ 32  32  30]
   [ 31  31  29]]

  ...

  [[ 30  37  43]
   [ 33  40  46]
   [ 33  40  46]
   ...
   [214 170 133]
   [213 169 132]
   [210 166 129]]

  [[ 29  36  42]
   [ 32  39  45]
   [ 34  41  47]
   ...
   [205 161 124]
   [206 162 125]
   [205 161 124]]

  [[ 31  38  44]
   [ 32  39  45]
   [ 34  41  47]
   ...
   [206 162 125]
   [210 166 129]
   [213 169 132]]]


 [[[ 50  52  51]
   [ 55  57  56]
   [ 58  60  59]
   ...
   [ 49  51  50]
   [ 50  52  51]
   [ 48  50  49]]

  [[ 52  54  53]
   [ 54  56  55]
   [ 56  58  57]
   ...
   [ 49  51  50]
   [ 49  51  50]
   [ 47  49  48]]

  [[ 54  56  55]
   [ 54  56  55]
   [ 53  55  54]
   ...
   [ 48  50  49]
   [ 48  50  49]
   [ 46  48

In [5]:

# Normalize image data to values between 0 and 1
images = images.astype('float32') / 255.0

# Encode class labels as integers
unique_labels = np.unique(labels)
label_to_int = {label: i for i, label in enumerate(unique_labels)}
int_labels = np.array([label_to_int[label] for label in labels])

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, int_labels, test_size=0.2, random_state=42)

#Building the MLP Model
model=models.Sequential([
    layers.Flatten(input_shape=(X_train.shape[1:])),
    layers.Dense(128,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(len(unique_labels), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model training using the training data
epochs = 500
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)


Epoch 1/500
18/18 [==============================] - 41s 2s/step - loss: 71.9551 - accuracy: 0.0699
Epoch 2/500
18/18 [==============================] - 38s 2s/step - loss: 15.8145 - accuracy: 0.1308
Epoch 3/500
18/18 [==============================] - 40s 2s/step - loss: 3.7972 - accuracy: 0.3799
Epoch 4/500
18/18 [==============================] - 43s 2s/step - loss: 2.4095 - accuracy: 0.4552
Epoch 5/500
18/18 [==============================] - 38s 2s/step - loss: 1.6221 - accuracy: 0.5950
Epoch 6/500
18/18 [==============================] - 39s 2s/step - loss: 1.2228 - accuracy: 0.6631
Epoch 7/500
18/18 [==============================] - 39s 2s/step - loss: 1.1853 - accuracy: 0.6613
Epoch 8/500
18/18 [==============================] - 38s 2s/step - loss: 1.6205 - accuracy: 0.6057
Epoch 9/500
18/18 [==============================] - 41s 2s/step - loss: 0.9692 - accuracy: 0.7079
Epoch 10/500
18/18 [==============================] - 40s 2s/step - loss: 0.7677 - accuracy: 0.7688
Epoch 1

In [5]:
test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", test_accuracy)

NameError: name 'model' is not defined

The model scored an accuracy of more than 80%!

In [4]:
# Save the trained model
model.save("hand_gesture_mlp_model.h5")

# Load the saved model
loaded_model = models.load_model("hand_gesture_mlp_model.h5")

# Function to predict the class of a particular image
def predict_image_class(image_path):
    img = Image.open(image_path)
    img = np.array(img).astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    predicted_probs = loaded_model.predict(img)
    predicted_class_idx = np.argmax(predicted_probs)
    predicted_class = unique_labels[predicted_class_idx]
    return predicted_class

# Making prediction using saved model with a test image
test_image_path = '/content/data/AllImage/i2.jpg'
predicted_class = predict_image_class(test_image_path)
print("Predicted class:", predicted_class)


NameError: name 'model' is not defined

My Model made correct prediction about the class of the image.✔☝

In [2]:
import pickle
pickle.dump(loaded_model,open('imodel.pkl','wb'))
pickled_model=pickle.load(open('imodel.pkl','rb'))

NameError: name 'loaded_model' is not defined